In [27]:
## Missed Trash Pickups

##In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

##As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. 
    ##One category of liquidated damages is related to chronic problems in any category of service at the same premises. 
    ##A chronic problem is defined as more than one missed pickup for any address. 
    ##The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

##Your job is to determine the total amount of damages due to missed pickups. 
##Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

##After determining the total amount of damages, you can look at other questions:


##* What other types of complaints are there?

##* Make a heat map that shows the most total missed pick ups and another that shows the total fines, each by zip code.

##* How do metro crews compare to the contractor's performance?

##* How much does each trash hauler owe?

##* What were to total missed pickup by route?



##Before you begin, explore the data.  Cleaning and preparing the data for analysis is an important and necessary step.  
##Planning and communication are vital to success. This data and analysis are based on a real-world project.
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point
import matplotlib.pyplot as plt
## import seaborn as sns
import folium

In [28]:
pickup_report = pd.read_csv('data/trash_hauler_report.csv')

In [29]:
                               # Fill missing fields 
pickup_report['Trash Hauler'] = pickup_report['Trash Hauler'].fillna('N/A')
pickup_report['Trash Route'] = pickup_report['Trash Route'].fillna('N/A')
pickup_report['Trash Route'] = pickup_report['Trash Route'].fillna('0')

In [30]:
pickup_report['Trash Route'] = pickup_report['Trash Route'].astype(str)
pickup_report['Trash Route'] = pickup_report['Trash Route'].str.replace('S', '', regex=False)
pickup_report['Trash Route'] = pd.to_numeric(pickup_report['Trash Route'], errors='coerce').fillna(0).astype(int)
print(f"Changed To 2303 instead of 2303S \n ----> {pickup_report.loc[43, 'Trash Route']}")

Changed To 2303 instead of 2303S 
 ----> 2303


In [31]:
pickup_report['Zip Code'] = pickup_report['Zip Code'].fillna(0).astype('category')
assert pickup_report['Zip Code'].dtype == 'category'

print(pickup_report['Zip Code'].describe())

display(pickup_report['Zip Code'])

count     20226.0
unique       29.0
top       37013.0
freq       2278.0
Name: Zip Code, dtype: float64


0        37207.0
1        37218.0
2        37209.0
3        37207.0
4        37209.0
          ...   
20221    37013.0
20222    37206.0
20223    37214.0
20224    37013.0
20225    37217.0
Name: Zip Code, Length: 20226, dtype: category
Categories (29, float64): [0.0, 37013.0, 37027.0, 37076.0, ..., 37219.0, 37220.0, 37221.0, 37228.0]

In [32]:
# Clean up column names and Normalize data
pickup_report = pickup_report.rename(columns={'Request ': 'Request'})
pickup_report['Trash Hauler'] = pickup_report['Trash Hauler'].str.upper()

In [33]:
# Create a dataframe made up of ONLY 'missed' appearing in the 'Request" column
missedpu = pickup_report[pickup_report['Request'].str.contains(r'\bmissed\b', case=False, na=False, regex=True)]
print (missedpu.describe())

       Request Number   Trash Route  Council District  State Plan X  \
count    15028.000000  15028.000000      14991.000000  1.500700e+04   
mean    151361.267234   4077.764906         18.318524  1.766425e+06   
std      71901.537542   2670.787558         10.107257  8.137032e+05   
min      25274.000000      0.000000          1.000000  1.663490e+06   
25%      87142.250000   2401.000000          8.000000  1.728654e+06   
50%     148228.500000   3401.000000         20.000000  1.747701e+06   
75%     220744.500000   4409.000000         27.000000  1.760562e+06   
max     267137.000000   9508.000000         35.000000  3.496892e+07   

       State Plan Y  
count  1.500700e+04  
mean   6.592279e+05  
std    4.550526e+04  
min   -2.719106e+05  
25%    6.395148e+05  
50%    6.570097e+05  
75%    6.757087e+05  
max    2.204382e+06  
